# Document Reranking Demo

Document reranking improves retrieval quality by using a specialized reranking model to reorder initial retrieval results. Rather than relying solely on semantic similarity from vector embeddings, reranking applies a more sophisticated relevance model (such as Cohere's rerank model) to score retrieved documents and return only the most relevant ones. This post-processing step refines results and reduces noise in the retrieved context.

## What this notebook contains
- Loading and chunking documents from web sources.
- Creating a vector database with OpenAI embeddings for initial retrieval.
- Setting up a base retriever to fetch top-k candidate documents.
- Using Cohere's reranking model to score and reorder retrieved documents.
- Building a `ContextualCompressionRetriever` that combines initial retrieval with reranking.
- Executing queries to retrieve and rerank documents based on relevance.

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from langchain_community.document_loaders import WebBaseLoader  
from langchain_community.vectorstores import Chroma  
from langchain_core.output_parsers import StrOutputParser  
from langchain_core.runnables import RunnablePassthrough  
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from langchain_community.llms import Cohere
from langchain.retrievers import  ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
import numpy as np
import yaml
import bs4  
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Get the current working directory
cwd = os.getcwd()

# Build the path to config.yaml
config_path = os.path.join(cwd, '..', 'configs', 'config.yaml')

# Normalize the path
config_path = os.path.abspath(config_path)

# Load credential from config file
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Set environment variables
os.environ['LANGCHAIN_API_KEY'] = config['API']['LANGCHAIN']
os.environ['OPENAI_API_KEY'] = config['API']['OPENAI']
os.environ['COHERE_API_KEY'] = config['API']['COHERE']

# Configure chat LLM (deterministic)
llm = ChatOpenAI(temperature=0) 

In [3]:
# Create a loader that fetches and parses the target web page
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # tuple of URLs to load
    bs_kwargs=dict(  # pass BeautifulSoup-specific kwargs to limit parsing
        parse_only=bs4.SoupStrainer(  # only parse these parts of the page to reduce noise
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Fetch and return a list of Document objects
docs = loader.load()  

# Split long documents into smaller overlapping chunks suitable for embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)  # list of smaller document chunks

# Create embeddings and store them in a vector DB (Chroma)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())  # uses OpenAI embeddings under the hood

# Create a retriever to fetch relevant docs (return the top 10 results)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [4]:
# Build a compression retriever using CohereRerank
compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Example query to retrieve relevant document
question = "What is task decomposition for LLM agents?"

# Retrieve compressed documents
compressed_docs = compression_retriever.get_relevant_documents(question)
compressed_docs 

/var/folders/2n/7qg7n4m55nn6p0lx1w8bgmqh0000gn/T/ipykernel_69246/702759450.py:2: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(model="rerank-english-v3.0")
/var/folders/2n/7qg7n4m55nn6p0lx1w8bgmqh0000gn/T/ipykernel_69246/702759450.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(question)


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'relevance_score': 0.99843913}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'relevance_score': 0.97467697}, page_content='Challenges in long-term planning and task decomposition: Planning over a lengthy history and effectively exploring the solution space remain challenging. LLMs struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'relevance_score': 0.8441522}, page_content='Component One: Planning#\nA complicated task usuall